# Face Recognition Using Facenet,Deep Learning &OpenCV

# Extract Embeddings from Face Dataset

Imports Libraries


In [1]:
import os
import cv2
import pickle
import argparse
import numpy as np
from imutils import paths

Construct the argument and parse the argumens

In [2]:
ap=argparse.ArgumentParser()

ap.add_argument("--dataset",required=True,help="path of dataset")
ap.add_argument("--embeddings",required=True,help="path of output data of facial embeddings")
ap.add_argument("--faceModel",required=True,help="path for face detection model")
ap.add_argument("--prototxt",required=True,help="path of prototxt model")
ap.add_argument("--embeddingModel",required=True,help="path of embedding model")

args=vars(ap.parse_args(["--dataset",r"C:\Users\SRKT\Desktop\opencv-face-recognition\dataset\Test",
                        "--embeddings",r"C:\Users\SRKT\Desktop\opencv-face-recognition\output",
                        "--faceModel",r"C:\Users\SRKT\Desktop\opencv-face-recognition\pre\face_detection_model\res10_300x300_ssd_iter_140000.caffemodel",
                        "--prototxt",r"C:\Users\SRKT\Desktop\opencv-face-recognition\pre\face_detection_model\deploy.prototxt",
                        "--embeddingModel",r"C:\Users\SRKT\Desktop\opencv-face-recognition\pre\openface_nn4.small2.v1.t7",
                        ]))


Loading Model

In [3]:
#loading face_detection model
#Caffe based DL face detector to localize faces in an image.
face_detector=cv2.dnn.readNetFromCaffe(args["prototxt"],args["faceModel"])

#loading face Embedding model
#This model is Torch-based and is responsible for extracting facial embeddings via deep learning feature extraction.
face_embedder=cv2.dnn.readNetFromTorch(args["embeddingModel"])

Loading Dataset

In [4]:
imagePaths=list(paths.list_images(args["dataset"]))

#initialize our lists of extracted facial embedding & corresponidng name
knownEmbeddings=[]
knownNames=[]

#initialized the total no of faces processed 
total=0


In [5]:
#loop over the image paths
for (i,imagePath) in enumerate(imagePaths):
    
    #extract person name from image path
    print("processing image {}/{}".format(i+1,len(imagePaths)))
    name=imagePath.split(os.path.sep)[-2]
    
    #load the image , resize image and grab the image dimensions
    image=cv2.imread(imagePath)
    image=cv2.resize(image,(650,600))
    (h,w)=image.shape[:2]
    
    #construct a blob from the image
    imageBlob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0),swapRB=False,crop=False)
    
    #apply opencv deep learning-based face detector to localize faces in the input image
    face_detector.setInput(imageBlob)
    detections=face_detector.forward()
    
    #ensure atlest one face is detected 
    if len(detections)>0:
        #making assumption that each image has only one face , so find bounding box with largest probability
        i=np.argmax(detections[0,0,:,2])
        confidence=detections[0,0,i,2]
        
        #ensure that the detections with largests probability also have min >0.5
        if confidence>0.5:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (start_x,start_y,end_x,end_y)=box.astype("int")
            
            #extract the face ROI and grab the ROI dimensions
            face=image[start_y:end_y,start_x:end_x]
            (fH,fW)=face.shape[:2]
            
            #ensure that face width & height are sufficiently large
            if fW<20 or fH<20:
                continue
            
            #construct the blob for the face ROI , then pass the blob through our face embedding model to
            #obtained the 128-d quantification of the face
            faceBlob=cv2.dnn.blobFromImage(face,1.0/255,(96,96),(0,0,0),swapRB=True,crop=False)
            face_embedder.setInput(faceBlob)
            vec=face_embedder.forward()
            
            #add the name of the person & corresponding face embedding to their respective list
            knownNames.append(name)
            knownEmbeddings.append(vec.flatten())
            total +=1

processing image 1/250
processing image 2/250
processing image 3/250
processing image 4/250
processing image 5/250
processing image 6/250
processing image 7/250
processing image 8/250
processing image 9/250
processing image 10/250
processing image 11/250
processing image 12/250
processing image 13/250
processing image 14/250
processing image 15/250
processing image 16/250
processing image 17/250
processing image 18/250
processing image 19/250
processing image 20/250
processing image 21/250
processing image 22/250
processing image 23/250
processing image 24/250
processing image 25/250
processing image 26/250
processing image 27/250
processing image 28/250
processing image 29/250
processing image 30/250
processing image 31/250
processing image 32/250
processing image 33/250
processing image 34/250
processing image 35/250
processing image 36/250
processing image 37/250
processing image 38/250
processing image 39/250
processing image 40/250
processing image 41/250
processing image 42/250
p

save the facial embedding & name

In [6]:
data={"embedding":knownEmbeddings,"names":knownNames}
f=open(r"C:\Users\SRKT\Desktop\opencv-face-recognition\output\Test\embeddings.pickle","wb")
f.write(pickle.dumps(data))
f.close()

In [7]:
len(knownEmbeddings)

247